In [ ]:
# @REMOVE-FROM-TEMPLATE
from plaster.tools.ipynb_helpers.displays import restart_kernel; restart_kernel()

In [ ]:
# @IMPORT-MERGE
import numpy as np
import pandas as pd
import itertools
from plaster.tools.log.log import error, debug
from plaster.run.job import JobResult
from plaster.run.run import RunResult
from plaster.run.plots import plots_dev
from plaster.run.sigproc_v2 import sigproc_v2_worker as worker
from plaster.tools.image.coord import WH, XY, roi_shift, clip2d
from plaster.tools.zplots import zplots
from plaster.tools.schema import check
from plaster.tools.image import imops
from plaster.tools.zap import zap
from plaster.tools.utils import utils
from plaster.tools.calibration.calibration import Calibration
from plaster.run.sigproc_v2.sigproc_v2_params import SigprocV2Params
from plaster.tools.ipynb_helpers.displays import hd
z = zplots.setup()

In [ ]:
# @REMOVE-FROM-TEMPLATE
from plumbum import local
job = JobResult("/erisyon/internal/jobs_folder/abbe2_1")
run = job.runs[0]

# PSF

In [ ]:
for ch_i in range(run.ims_import.n_channels):
    hd("h2", f"Channel {ch_i}")
    psfs = run.sigproc_v2.calib.psfs(ch_i=ch_i)

    with z(_noaxes=True):
        display("Below:")
        with z(_cols=6):
            n_zs = psfs.shape[0]
            for z_i in range(0, n_zs // 2):
                plots_dev.plot_psfs(psfs[z_i], scale=2.0, f_title=f"ch_i={ch_i}, z_i={z_i}", _zplots_context=z)

        display("Most in Focus:")
        z_i = n_zs // 2
        plots_dev.plot_psfs(psfs[z_i], scale=2.0, f_title=f"ch_i={ch_i}, z_i={z_i}", _zplots_context=z, _noaxes=True, _notools=True)

        display("Above:")
        with z(_cols=6):
            for z_i in range(n_zs // 2 + 1, n_zs):
                plots_dev.plot_psfs(psfs[z_i], scale=2.0, f_title=f"ch_i={ch_i}, z_i={z_i}", _zplots_context=z)

# Illumination Balance

In [ ]:
for ch_i in range(run.ims_import.n_channels):
    with z(_cols=6):
        key = f"regional_illumination_balance.instrument_channel[{ch_i}]"
        val = run.sigproc_v2.calib.get(key)
        if val is not None:
            hd("h2", f"Channel {ch_i}")
            bal = np.array(val)
            z.im(1.0 / bal)
        else:
            print(f"No illumination balance on channel {ch_i}")